In [1]:
import requests
import pandas as pd
import numpy as np
import json
stop_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-stops&q=&rows=799"
driveway_api_url = "https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=driveways&q=&rows=-1&facet=material"
busroute_api_url = 'https://opendatakingston.cityofkingston.ca/api/records/1.0/search/?dataset=transit-gtfs-routes&q=&rows=-1&facet=route_short_name&exclude.route_long_name=999+Maintenance+-+Out+of+Service'
stop_response = requests.get(stop_api_url)
driveway_response = requests.get(driveway_api_url)
busroute_response = requests.get(busroute_api_url)

In [2]:
driveway_response.status_code

200

In [3]:
x = stop_response.json()
x_driveway = driveway_response.json()
x_busroute = busroute_response.json()

In [40]:
transit_stops = pd.json_normalize(x,'records')
driveways = pd.json_normalize(x_driveway,'records')
busroute = pd.json_normalize(x_busroute,'records')

In [54]:
# drops data if there is no GPS coordinates.
driveways = driveways.dropna(subset = ['fields.geo_point_2d'],how = 'all')

In [6]:
import math

def haversine(coord1,coord2):
    # returns shortest distance between two points in metres
    
    # Radius of the Earth at Kingston. Assuming mean elevation of 93m above
    # sea level.
    R = 6367869
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a),math.sqrt(1-a))

In [7]:
haversine(np.array(driveways['fields.geo_point_2d'][0]),np.array(transit_stops['fields.stop_coordinates'][0]))

1659.8080744442598

In [8]:
transit_stops['bus_routes'] = pd.Series()

c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


The next section of code is trying to determine which routes service which stops. It is using map shading GPS coords of the bus routes to measure the distance of the route to the stop. A stop is said to be part of a bus route if the haversine formula measures a distance of \<200m of the shaded route to the stop. The shape parameter which marks the GPS coordinates of the bus routes are range from 20m - 300m between points. 

In [9]:
# The calculation needs to be complete for all bus stops. Stops need to be recorded in the dataframe. TBD.
k = 0
for s in transit_stops['fields.stop_coordinates']:
    stop_test = np.array(s)
    route_list = list()
    for route, ind in enumerate(busroute['fields.shape.coordinates']):
        p = 0
        route_test = np.array(busroute['fields.shape.coordinates'][route][0])
        for ind in route_test:
            test_param = haversine(np.flip(route_test[p]),stop_test)
            if test_param<150:
                # Append bus route to list.
                route_list.append(busroute['fields.route_short_name'][route])
                # Loop breaks if stop identified as being within 100m of bus route.
                break


            p = p+1

    transit_stops['bus_routes'][k] = route_list
    print(transit_stops['fields.stop_name'][k]," ",transit_stops['bus_routes'][k])
    k = k+1



c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\ryan\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


Norwest Road (east side of Midland)   ['6', '15', '7', '14', '501', '4', '10', 'COV']
Centennial Drive (south side of Cataraqui Woods)   ['14']
Crossfield Avenue (west side of Andersen)   ['14']
Chadwick Parkette (west side of Andersen)   ['14']
Vanguard Court (east side of Waterloo)   ['14']
Centennial Drive (north side of Davis)   ['14']
Bigham Court (south side of Davis)   ['14']
Trillium Ridge (west side of Edgar)   ['14']
Bexley Gate (north side of Taylor-Kidd)   ['14']
RioCan Centre (north side of Taylor-Kidd)   ['14']
Blackburn Mews (north side of Taylor-Kidd)   ['14']
Kimberley Street (east side of Old Colony)   ['14']
Ridley Drive (east side of Old Colony)   ['15', '14']
Sutton Mills Court (east side of Ridley)   ['6', '15', '7', '14', '501', '4', '10', 'COV']
Norwest Road (east side of Ridley)   ['15', '14']
257 Bath Road (north side)   ['16', '3', '11']
258 Queen Mary Road (west side)   ['3', '11']
Queen Mary Road (south side of Robert Wallace)   ['3']
Curtis Crescent (west 

Francis Street (west side of Mowat)   ['2']
Cartwright Street (north side of Churchill)   ['6', '2', '3']
Albert Street (south side of Union)   ['2']
Douglas Avenue (north side of Elliott)   ['7', '16', '2']
Kingslake Plaza (south side of Benson)   ['16', '2']
300 Conacher Drive (south side)   ['16', '2']
230 Conacher Drive (south side)   ['7', '16', '2']
Wilson Street (south side of Conacher)   ['7', '16', '2']
Sutherland Drive (east side of Conacher)   ['2']
Sutherland Drive (east side of Virginia)   ['1', '7', '2']
Mgr. Remi-Gaulin School (south side of Virginia)   ['1', '7', '2']
Esdon Street (east side of Guthrie)   ['1', '7', '2']
Seniors Centre (east side of Francis)   ['2']
1130 Montreal Street (west side)   ['801']
Daly Street (north side of Weller)   ['801']
Baker Street (north side of Weller)   []
71 Compton Street (north side)   ['7', '16', '2']
Weller Avenue at KCHC (west side of Compton)   ['7', '16']
Wiley Street (south side of Drennan)   []
Wilson Street (south side of 

Gardiners Road (south side of Cataraqui Woods)   ['7', '14']
McMichael Street (south side of Glengarry)   []
Seventh Avenue (east side of Kingscourt)   ['16']
Oak Street (east side of Kingscourt)   ['16']
Kingscourt Avenue (south side of Kirkpatrick)   ['16']
Theresa Crescent (north side of Kirkpatrick)   ['702', '16', '2', '701']
Kingscourt Avenue (north side of Kirkpatrick)   ['16']
Third Avenue (west side of Kingscourt)   ['16']
Concession Street (west side of Kingscourt)   ['16']
Macdonnell Street (north side of Concession)   ['16']
Leroy Grant Drive (north side of Concession)   ['16']
Glen Castle Road (south side of Henderson)   ['15']
Portsmouth Avenue (south side of King)   ['6', '2', '3']
1055 Princess Street/Kingston Centre (north side)   ['501', '4', '502']
1281 Midland Avenue (west side)   ['15']
321 Yonge Street (east side)   ['2']
Cataraqui Centre Transfer Point Platform 5   ['6', '15', '7', '14', '501', '4', '10', '11', '502', 'COV']
Cataraqui Centre Transfer Point Platfo

425 Knightsbridge Road (west side)   ['15']
Glen Castle Road (west side of Chelsea)   ['15']
Taylor-Kidd Boulevard (south side of Princess)   ['7', '14', '16', '501', '4', '502']
Portsmouth Avenue (south side of Princess)   ['16', '501', '4', '502']
Heathfield (south side of Princess)   ['501', '4', '502']
800 Princess Street (south side)   ['501', '4', '502']
Alfred Street (south side of Princess)   ['501', '4', '502']
Clergy Street (north side of Brock)   ['801', '12', '702', '501', '2', '4', '502', '701']
Legion Villa (north side of Princess)   ['501', '4', '502']
Westdale Avenue (north side of Bath)   ['12', '16', '501', '4', '502']
Sir John A Macdonald Blvd (north side of Princess)   ['501', '4', '502']
1225 Princess Street (north side)   ['501', '4', '502']
McMichael Street (north side of Princess)   ['501', '4', '502']
John Counter Boulevard (north side of Princess)   ['7', '14', '16', '501', '4', '502']
RioCan Centre (west side of driveway)   ['11']
1600 Bath Road (south side) 

Parkway (north side of Princess)   ['16', '501', '4', '502']
Andersen Drive (north side of Princess)   ['7', '14', '501', '4', '502']
940 Gardiners Road (west side)   ['7', '14', '501', '4', '502']
Taylor-Kidd Boulevard (west side of Gardiners)   ['14', '11']
RioCan at Petsmart (south side of driveway)   ['11']
Arbour Heights (east side of Tanner)   ['11']
Kingston Centre Transfer Point Platform 5   ['12', '16', '501', '2', '3', '4', '11']
Queen Mary Road (north side of Bath)   ['3', '11']
Owens Crescent (north side of Kingsdale)   ['11']
259 Kingsdale Avenue (north side)   ['11']
Dolshire Street (north side of Malabar)   ['11']
1600 Bath Road (north side)   ['11']
RioCan at Best Buy (north side of driveway)   ['11']
First Canada Avenue (west side of Division)   ['702', '7', '16', '2', '701']
Victoria Street (north side of Brock)   ['12']
Frontenac Mall (north side of Bath)   []
Tanner Drive (north side of Bath)   ['11']
Centennial Drive (south side of Bath)   []
324 Palace Road (west 

The next bit of code was used to investigate the distance between shape GPS coords. There is an index error when it reaches the end of the loop. To look at other bus routes add +int to indexing \[route\] variable.

In [55]:
for route, ind in enumerate(busroute['fields.shape.coordinates']):
    p = 0
    route_test = np.array(busroute['fields.shape.coordinates'][route][0])
    testparam = list()
    for ind in route_test:
            testparam.append(haversine(np.flip(route_test[p]),np.flip(route_test[p+1])))
            p = p+1

IndexError: index 71 is out of bounds for axis 0 with size 71

In [14]:
testparam.sort()
mid = len(testparam)//2
print("median distance between shape coods: ",(testparam[mid]+testparam[~mid])//2)
print("mean distance between shape coods: ",(sum(testparam)//len(testparam)))

median distance between shape coods:  103.0
mean distance between shape coods:  90.0


In [56]:
transit_lat = transit_stops['fields.stop_coordinates'].explode().to_numpy()[::2]
transit_long = transit_stops['fields.stop_coordinates'].explode().to_numpy()[1::2]
transit_coords = np.column_stack((transit_lat,transit_long))

driveway_lat = driveways['fields.geo_point_2d'].explode().to_numpy()[::2]
driveway_long =  driveways['fields.geo_point_2d'].explode().to_numpy()[1::2]

driveway_coords = np.column_stack((driveway_lat,driveway_long))

In [57]:
stop_distance = np.ndarray(shape = [driveway_lat.size,transit_lat.size])
k = 0
l = 0

for i in driveway_coords:
    l = 0
    for j in transit_coords:
        stop_distance[k,l] = haversine(i,j)
        l = l+1
    k = k+1

In [58]:
closest_stop = stop_distance.min(axis = 1)
print('median distance to stop: ',np.median(closest_stop),'m')
print(closest_stop.mean())

median distance to stop:  195.5921455688095 m
1088.1638232910855


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=63c92067-2a93-4cc0-850b-51f44255f10b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

We can evaluate the October ridership transit data provided by the city of Kingston as an estimate for the daily usage of the transit network. We can see if there is a correlation between ridership and distance from driveways to bus stops.